In [1]:
pip install streamlit tensorflow tensorflow-hub matplotlib numpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
streamlit_code = """
import streamlit as st
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
from PIL import Image
import io

# Load the Pose Detection Model
@st.cache_resource
def load_model():
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    return model

model = load_model()

# Function to Detect Pose
def detect_pose(image_file):
    try:
        # Read the image as raw binary data
        image_bytes = image_file.read()

        # Convert binary data to a PIL Image
        image = Image.open(io.BytesIO(image_bytes))
        
        # Convert PIL image to a NumPy array and prepare it for model
        image_np = np.array(image)
        image_rgb = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

        # Resize the image and process it for model input
        input_image = tf.image.resize_with_pad(image_rgb, 192, 192)
        input_image = tf.expand_dims(input_image, axis=0)
        input_image = tf.cast(input_image, dtype=tf.int32)

        # Run the model
        outputs = model.signatures["serving_default"](input_image)
        keypoints = outputs["output_0"].numpy()[0, 0, :, :]
        
        # Debugging: Print the shape and content of the keypoints
        print(f"Keypoints shape: {keypoints.shape}")
        print(f"Keypoints data: {keypoints}")

        return keypoints, image_rgb

    except Exception as e:
        st.error(f"Error during pose detection: {e}")
        return None, None

# Function to Classify Pose
def classify_pose(keypoints):
    try:
        # Extract relevant keypoints (e.g., nose, shoulders, elbows, etc.)
        nose = keypoints[0]
        left_shoulder = keypoints[5]
        right_shoulder = keypoints[6]
        left_hip = keypoints[11]
        right_hip = keypoints[12]
        left_knee = keypoints[13]
        right_knee = keypoints[14]
        left_ankle = keypoints[15]
        right_ankle = keypoints[16]

        # Example: Tree Pose (Standing with one leg raised)
        if left_hip[2] > 0.3 and right_hip[2] > 0.3:  # Confidence threshold for standing pose
            return "Tree Pose"

        # Example: Downward Dog Pose (Body angled with arms and legs forming an upside-down V)
        elif left_shoulder[2] > 0.5 and right_shoulder[2] > 0.5:  # Confidence threshold for arms up
            return "Downward Dog Pose"

        # Example: Warrior Pose (Lunging forward with arms raised)
        elif left_knee[2] > 0.5 and right_knee[2] > 0.5:  # Confidence threshold for knees bent
            if left_shoulder[2] > 0.6 and right_shoulder[2] > 0.6:  # Arms raised
                return "Warrior Pose"

        # Example: Child's Pose (Body bent forward, knees wide, arms stretched forward)
        elif left_knee[2] > 0.5 and right_knee[2] > 0.5:
            if nose[2] < 0.2:  # Nose at lower part of the body (head down)
                return "Child's Pose"

        # Example: Cobra Pose (Lying on the stomach with arms pushing up)
        elif left_shoulder[2] > 0.5 and right_shoulder[2] > 0.5:
            if left_ankle[2] < 0.2 and right_ankle[2] < 0.2:  # Feet flat on the ground
                return "Cobra Pose"

        # Default fallback if no pose is detected
        return "Pose Not Recognized"
        
    except Exception as e:
        st.error(f"Error during pose classification: {e}")
        return "Error in Classification"

# Display results
def display_results(image, keypoints):
    try:
        if image is None or keypoints is None:
            st.write("No image or keypoints to display.")
            return
        
        h, w, _ = image.shape
        pose_detected = False
        
        # Draw keypoints and connections for visualization
        for kp in keypoints:
            y, x, confidence = kp
            if confidence > 0.3:  # Lower confidence threshold for testing
                pose_detected = True
                cv2.circle(image, (int(x * w), int(y * h)), 5, (0, 255, 0), -1)

        if not pose_detected:
            st.write("No valid pose detected with confidence above threshold.")

        # Convert back to PIL for display
        annotated_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Display Image and Pose
        st.image(annotated_image, caption="Detected Pose", use_column_width=True)

        # Classify the pose
        pose_name = classify_pose(keypoints)
        st.write(f"Detected Pose: {pose_name}")

    except Exception as e:
        st.error(f"Error during image display: {e}")

# Streamlit UI
st.title("Yoga Pose Detection")
st.write("Upload an image of your yoga pose to get feedback.")

uploaded_file = st.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    keypoints, original_image = detect_pose(uploaded_file)
    display_results(original_image, keypoints)

"""

# Save the Streamlit code to a Python file
with open('yoga_pose_detection.py', 'w') as f:
    f.write(streamlit_code)

print("Streamlit app code saved to 'yoga_pose_detection.py'")


Streamlit app code saved to 'yoga_pose_detection.py'


In [ ]:
import subprocess

# Run the Streamlit app
subprocess.run(["streamlit", "run", "yoga_pose_detection.py"])


In [ ]:
!streamlit run yoga_pose_detection.py

In [ ]:
streamlit run yoga_pose_detection.py --safe
